In [212]:
import re
from collections import deque
from collections.abc import Iterable
from typing import Union

In [131]:
class bcolors:
  HEADER = '\033[95m'
  OKBLUE = '\033[94m'
  OKCYAN = '\033[96m'
  OKGREEN = '\033[92m'
  WARNING = '\033[93m'
  FAIL = '\033[91m'
  ENDC = '\033[0m'
  BOLD = '\033[1m'
  UNDERLINE = '\033[4m'

In [332]:
class CalculatorError(Exception):
    """Base class for Calculator exceptions"""
    pass

class ParenthesesBalancingError(CalculatorError):
    """Raised when unbalanced parentheses are found in input"""
    def __init__(self, input_string):
        self.input_string = input_string
        message = f"Matching parenthesis not found in input \'{self.input_string}\'."
        super().__init__(message)

class UnknownToken(CalculatorError):
    """Raised when an unknown token is present in math expression"""
    def __init__(self, token):
        self.unknown_token = token
        message = f"Couldn't understand token \'{self.unknown_token}\'."
        super().__init__(message)

class IncompleteExpression(CalculatorError):
    def __init__(self, operator):
        self.operator = operator
        message = f"No operand found for operator \'{self.operator}\'."
        super().__init__(message)

In [333]:
class Calculator:
    precedence_levels = {
        "^": 9,
        #'-' : 9
        "*": 8,
        "/": 8,
        "%": 8,
        "+": 6,
        "-": 6,
        "(": -1,
        ")": None,
    }

    operators = list(filter(lambda x: x not in ["(", ")"], precedence_levels.keys()))

    funcs = {
        "+": (lambda a, b: a + b),
        "-": (lambda a, b: a - b),
        "*": (lambda a, b: a * b),
        "/": (lambda a, b: a / b),
        "^": (lambda a, b: a ** b),
        "%": (lambda a, b: a % b),
    }

    def __init__(self) -> None:
        self.history = []

    def get_char_type(self, char) -> str:
        """
        Returns a string denoting the type of char.
        """
        #operators = {"*", "/", "+", "-", "%", "/"}
        if char.isdigit() or char == ".":
            char_type = "number"
        elif char.isalpha():
            char_type = "letter"
        elif char in self.operators:
            char_type = "operator"
        else:
            char_type = "other"
        return char_type

    def tokenize(self, string) -> Iterable[str]:
        """
        Generates tokens from a mathematical statement string.
        """
        token_type = self.get_char_type(string[0])
        token = ''
        for char in re.sub(r'\s+', "", string):
            #if char == ' ':
            #    continue  # Spaces are not included
            new_type = self.get_char_type(char)
            if char == '-' and \
                (token_type == 'operator' or token[-1]=='('):
                yield token
                token_type = 'number'
                token = ''
            elif new_type != token_type:  # A new type of token has been found 
                yield token
                token_type = new_type
                token = ''
            token += char
        if len(token) > 0:
            yield token

    def is_float(self, element: str) -> bool:
        try:
            float(element)
            return True
        except ValueError:
            return False

    def int_or_float(self, number: str) -> Union[int, float]:
        try:
            return int(number)
        except ValueError:
            return float(number)

    def convert_to_rpn(self, input_string) -> list:
        output = []
        operator_stack = deque()

        for token in input_string:
            if self.is_float(token):
                output.append(token)

            elif token in self.operators:
                while True:
                    if len(operator_stack) == 0:
                        break
                    if self.precedence_levels[token] > self.precedence_levels[operator_stack[-1]]:
                        break

                    popped = operator_stack.pop()
                    output.append(popped)

                operator_stack.append(token)

            elif token == '(':
                operator_stack.append(token)

            elif token == ')':
                while True:
                    if len(operator_stack) == 0:
                        raise ParenthesesBalancingError(''.join(input_string))
                    if operator_stack[-1] == '(':
                        _ = operator_stack.pop()
                        break
                    
                    popped = operator_stack.pop()
                    output.append(popped)

            else:
                raise UnknownToken(token)

        while len(operator_stack) > 0:
            stack_popped = operator_stack.pop()
            if stack_popped == '(':
                raise ParenthesesBalancingError(''.join(input_string))
            
            output.append(stack_popped)

        return output

    def eval_rpn(self, rpn_expression: list) -> float:
        parsing_stack = deque()

        for token in rpn_expression:
            if self.is_float(token):
                parsing_stack.append(self.int_or_float(token))
            else:
                try:
                    arg2 = parsing_stack.pop()
                    arg1 = parsing_stack.pop()
                    result = self.funcs[token](arg1, arg2)
                    parsing_stack.append(result)
                except IndexError:
                    raise IncompleteExpression(token) from None

        eval_result = parsing_stack.pop()
        return eval_result

    def calculate(self, input_string: str) -> Union[int, float]:
        tokenized = [i for i in self.tokenize(input_string) if i != '']
        rpn = self.convert_to_rpn(tokenized)
        result = self.eval_rpn(rpn)

        return result

In [338]:
test_string = '2^10'
#answer = eval(test_string)
Calculator().calculate(test_string)

1024

In [340]:
eval('2**10')

1024

In [337]:
isinstance(5, float)

False

In [254]:
type(eval('-1.'))

float

In [3]:
print(f"{bcolors.WARNING}Warning: No active frommets remain. Continue?{bcolors.ENDC}")

In [110]:
test_string = '-3+5'
answer = 903

In [168]:
precedence_levels = {
    '^' : 9,
    '-' : 9, #!!!
    '*' : 8,
    '/' : 8,
    '%' : 8,
    '+' : 6,
    '-' : 6, #!!!
    '(' : -1,
    ')' : None
}

operators = list(filter(lambda x: x not in ['(', ')'], precedence_levels.keys()))

In [175]:
def is_float(element: str) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

In [174]:
is_float('-1')

True

In [162]:
input = list(test_string)
output = []
operator_stack = deque()

for token in input:
    if token.isdigit():
        output.append(token)

    elif token in operators:
        while True:
            if len(operator_stack) == 0:
                break
            if precedence_levels[token] > precedence_levels[operator_stack[-1]]:
                break

            popped = operator_stack.pop()
            output.append(popped)

        operator_stack.append(token)

    elif token == '(':
        operator_stack.append(token)

    elif token == ')':
        while True:
            if operator_stack[-1] == '(':
                _ = operator_stack.pop()
                break
            if len(operator_stack) == 0:
                raise ValueError('unbalanced (')
            
            popped = operator_stack.pop()
            output.append(popped)

    else:
        raise ValueError('final else')

while len(operator_stack) > 0:
    stack_popped = operator_stack.pop()
    if stack_popped == '(':
        raise ValueError('unbalanced (')
    
    output.append(stack_popped)

In [163]:
output

['3', '-', '5', '+']

In [113]:
parsing_stack = deque()

funcs = {
  "+": (lambda a, b: a + b),
  "-": (lambda a, b: a - b),
  "*": (lambda a, b: a * b),
  "/": (lambda a, b: a / b),
  "^": (lambda a, b: a ** b),
  "%": (lambda a, b: a % b)
}

for token in output:
    if token.isdigit():
        parsing_stack.append(int(token))
    else:
        arg2 = parsing_stack.pop()
        arg1 = parsing_stack.pop()
        result = funcs[token](arg1, arg2)
        parsing_stack.append(result)

eval_result = parsing_stack.pop()

IndexError: pop from an empty deque

In [114]:
eval_result = parsing_stack.pop()

IndexError: pop from an empty deque

In [115]:
eval_result

18